In [1]:
import pandas as pd
import numpy as np
from  graphviz import Source
from scipy import misc
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, hamming_loss
import matplotlib.pyplot as plt
import seaborn as sns

#### Importar dados para analise
Para iniciar a analise dos dados primeiro necessita-se
importar esse dados, os .csv encontrados abaixo são os dados disponibilizados
[aqui](https://archive.ics.uci.edu/ml/datasets/record+linkage+comparison+patterns).
Após carregar cada um dos dados .csv unimos em um unico frame do pandas.

In [2]:
dataframe = pd.DataFrame()
for x in range(1,11):
    dataframe_name = 'data/block_'+str(x)+'.csv'
    new_dataframe = pd.read_csv(dataframe_name)
    dataframe = pd.concat([dataframe,new_dataframe])

frame = dataframe

#### Exibir Frame
Exibir resultado do merge dos frames..

In [3]:
frame.head()

,id_1,id_2,cmp_fname_c1,cmp_fname_c2,cmp_lname_c1,cmp_lname_c2,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz,is_match
0,37291,53113,0.833333333333333,?,1.0,?,1,1,1,1,0,True
1,39086,47614,1,?,1.0,?,1,1,1,1,1,True
2,70031,70237,1,?,1.0,?,1,1,1,1,1,True
3,84795,97439,1,?,1.0,?,1,1,1,1,1,True
4,36950,42116,1,?,1.0,1,1,1,1,1,1,True


In [4]:
frame.tail()

,id_1,id_2,cmp_fname_c1,cmp_fname_c2,cmp_lname_c1,cmp_lname_c2,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz,is_match
574908,32517,73116,1,?,0.222222,?,1,0,1,0,0,False
574909,67707,83757,0.111111111111111,?,1.000000,?,1,0,0,0,0,False
574910,53258,91808,1,?,0.000000,?,1,0,0,1,0,False
574911,31865,85285,1,?,0.111111,?,1,0,1,0,0,False
574912,33119,76399,1,?,0.000000,?,1,0,1,0,0,False


#### Arrumando tipos de dados

In [5]:
broken_columns = list(frame.columns[2:11])

for column in broken_columns:
    frame[column] = frame[column].apply(lambda x: np.NaN if x == '?' else x)
    frame[column] = frame[column].apply(lambda x: float(x) if type(x) == str else x)

In [6]:
imp_values = frame.drop(['id_1','id_2'],axis=1,inplace=True)

#### Verificando a existência de valores nulos

In [ ]:
melted_frame = pd.melt(imp_values.notnull())

In [ ]:
plt.figure(figsize=(15,4))
sns.countplot(melted_frame['variable'],hue=melted_frame['value'])
plt.tight_layout()

No gráfico acima as barras em azul representam o número de valores nulos de cada coluna

In [ ]:
sns.heatmap(frame.drop(['id_1','id_2'],axis=1).isnull(),cbar=False,yticklabels=False)

Acima vemos outro gráfico que representa a quantidade de valores nulos de cada coluna. Nesse caso as partes claras
representam os valores faltantes

#### Removendo colunas vazias
Com o resultado da exibição do head e d tail do frame, observa-se a existencia de colunas
com dados nulos, para a utilização do do sklearn para a criação da arvore de decisão é 
necesario remover os dados em branco, ou no caso do dataset do problema os dados com valor '?',
para isso utilizamos o arquivo frequencies.cvs disponibilizado no dataset do problema, para fazer a substituição
dos valores nulos para a media dos valores da coluna.

Removendo colunas desnecessario para aplicar o modelo.

In [7]:
frame.drop(['cmp_fname_c2','cmp_lname_c2'],axis=1,inplace=True)
frame.head()

,cmp_fname_c1,cmp_lname_c1,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz,is_match
0,0.833333,1.0,1,1.0,1.0,1.0,0.0,True
1,1.000000,1.0,1,1.0,1.0,1.0,1.0,True
2,1.000000,1.0,1,1.0,1.0,1.0,1.0,True
3,1.000000,1.0,1,1.0,1.0,1.0,1.0,True
4,1.000000,1.0,1,1.0,1.0,1.0,1.0,True


Aplicando as medias para os demais valores faltantes dos frames.

In [8]:
def preparer_data(frame):
    frame["cmp_fname_c1"] = frame["cmp_fname_c1"].replace(np.NaN,0.000235404896421846)
    frame["cmp_lname_c1"] = frame["cmp_lname_c1"].replace(np.NaN,2.68694413843136e-05)
    frame["cmp_sex"] = frame["cmp_sex"].replace(np.NaN,0.5)
    frame["cmp_bd"] = frame["cmp_bd"].replace(np.NaN,0.032258064516129)
    frame["cmp_bm"] = frame["cmp_bm"].replace(np.NaN,0.0833333333333333)
    frame["cmp_by"] = frame["cmp_by"].replace(np.NaN, 0.00943396226415094)
    frame["cmp_plz"] = frame["cmp_plz"].replace(np.NaN, 0.000422654268808115)
    
    return frame

frame = preparer_data(frame)


#### Criação da coluna de alvos
Para conseguir classificar os dados de acordo com coluna de alvos,
onde o sklearn index valores numericos aos dados categoricos que deseja-se
obter a predição.

In [9]:
def make_target_frame(frame, target_column):
    df_mod = frame.copy()
    targets = df_mod[target_column].unique()
    map_to_int = {name: n for n, name in enumerate(targets)}
    df_mod["Target"] = df_mod[target_column].map({True:1, False:0})
    return (df_mod, targets)
df2, targets = make_target_frame(frame, "is_match")


#### Exibição dos dados do Frame de alvos


In [10]:
df2[["Target", "is_match"]].head()


,Target,is_match
0,1,True
1,1,True
2,1,True
3,1,True
4,1,True


In [11]:
df2[["Target", "is_match"]].tail()

,Target,is_match
574908,0,False
574909,0,False
574910,0,False
574911,0,False
574912,0,False


#### Verificando a existência de valores nulos

In [12]:
frame.isnull().values.any()

False

In [13]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5749132 entries, 0 to 574912
Data columns (total 8 columns):
cmp_fname_c1    float64
cmp_lname_c1    float64
cmp_sex         int64
cmp_bd          float64
cmp_bm          float64
cmp_by          float64
cmp_plz         float64
is_match        bool
dtypes: bool(1), float64(6), int64(1)
memory usage: 356.4 MB


Ainda para observar, se as demais features que irão realmente compor o modelo, iremos buscar a correlação entre elas,
casos duas features tenham correlação muito alta, deverão ser desconsideradas no modelo, pois elas basicamente estariam 
trazendo a mesma informação ao modelo.

In [14]:
frame.corr()

,cmp_fname_c1,cmp_lname_c1,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz,is_match
cmp_fname_c1,1.000000,-0.668145,0.151666,0.016742,0.274531,0.033493,0.031607,0.044168
cmp_lname_c1,-0.668145,1.000000,0.109754,-0.210898,-0.392630,-0.219032,0.101822,0.123234
cmp_sex,0.151666,0.109754,1.000000,-0.242386,-0.151130,-0.239898,0.008734,0.009416
cmp_bd,0.016742,-0.210898,-0.242386,1.000000,-0.183648,0.072568,0.085247,0.111908
cmp_bm,0.274531,-0.392630,-0.151130,-0.183648,1.000000,-0.181056,0.045027,0.061539
cmp_by,0.033493,-0.219032,-0.239898,0.072568,-0.181056,1.000000,0.086154,0.112324
cmp_plz,0.031607,0.101822,0.008734,0.085247,0.045027,0.086154,1.000000,0.776620
is_match,0.044168,0.123234,0.009416,0.111908,0.061539,0.112324,0.776620,1.000000


Partindo da tabela de correlação podemos intenficar, que os dados estão bem desacoplados, sendo assim podemos utilizalos no nosso modelo.

In [15]:
frame.head()

,cmp_fname_c1,cmp_lname_c1,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz,is_match
0,0.833333,1.0,1,1.0,1.0,1.0,0.0,True
1,1.000000,1.0,1,1.0,1.0,1.0,1.0,True
2,1.000000,1.0,1,1.0,1.0,1.0,1.0,True
3,1.000000,1.0,1,1.0,1.0,1.0,1.0,True
4,1.000000,1.0,1,1.0,1.0,1.0,1.0,True


In [16]:
frame.tail()

,cmp_fname_c1,cmp_lname_c1,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz,is_match
574908,1.000000,0.222222,1,0.0,1.0,0.0,0.0,False
574909,0.111111,1.000000,1,0.0,0.0,0.0,0.0,False
574910,1.000000,0.000000,1,0.0,0.0,1.0,0.0,False
574911,1.000000,0.111111,1,0.0,1.0,0.0,0.0,False
574912,1.000000,0.000000,1,0.0,1.0,0.0,0.0,False


#### Criação da visualização da árvore de decisão 
Seleciona-se as colunas do frame que farão parte da analise, para esse problema
pela comando da questão as duas primeiras colunas não iriam compor a analise,
após inicializa-se o treinamento do modelo com as colunas selecionadas em x e y,
prepara-se a visualização da árvore.

In [18]:
columns = list(df2.columns[0:7])
y = df2["Target"]
x = frame[columns]
tree = DecisionTreeClassifier()
tree.fit(x,y)

def visualize_tree(tree, feature_names):
    
    with open("tree.dot", 'w') as f:
        export_graphviz(tree, out_file=f,
                        feature_names=feature_names)

    command = ["dot", "-Tpng", "tree.dot", "-o", "tree.png"]
    try:
        subprocess.check_call(command)
    except:
        exit("Não foi possivel abrir o arquivo")
    
visualize_tree(tree, columns)

o resultado da visualização da arvore de decisão é a seguinte:

![DecisionTree](tree.png "DecisionTree")

#### Avaliando a precisão do modelo
Para avaliar a precisão do modelo os grupos de dados são divididos em dados de treino e dados de teste, onde os dados de treino são x_train e y_train, e os dados de teste são, x_test e y_test. Para realizar a separação de dados utiliza-se o método train_test_split(), da seguinte forma:

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

Após dividir os grupos de dados, sendo que os dados de teste são definidos em 33% e os dados de treino são 67% dos dados originais, então é realizado o treino dos dados utilizando o método fit(), a previsão dos dados de teste utilizando o método predict() e uma matriz de confusão dos dados de teste com o método confusion_matrix(), como em:

In [20]:
tree.fit(x_train, y_train)
preds = tree.predict(x_test)

Com os dados treinados e previstos, gera-se a matriz de confusão onde existem os dados true_negative(tn), false_positive(fp), false_negative(fn) e true_positive(tp):

In [21]:
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
(tn, fp, fn, tp)

(1890327, 10, 13, 6864)

Após a matriz de confusão, para a avaliação do modelo realiza-se a comparação dos dados de teste previstos e os dados classificados escolhidos aleatoriamente utilizando o método accuracy_score() e obtém-se a precisão da classificação:

In [22]:
print(accuracy_score(y_test, preds))

0.9999878769606381


O valor da precisão obtido é de 99,9% o que sugere que em 99,9% das vezes a árvore de decisão é capaz de prever se o "match" é verdadeiro ou falso. Após o accuracy_score() utiliza-se o método precision_score() para verificar se os dados de teste que atestaram verdadeiro foram previstos corretamente em relação a todos os dados que atestaram falso de forma incorreta:

In [23]:
print(precision_score(y_test, preds))

0.9985452429444283


É possível observar que a precisão atestada é de 99,85%, o que sugere que em 99,85% das vezes a árvore de decisão é capaz de prever os dados classificados como verdadeiros corretamente. Como mais uma forma de precisão utiliza-se o método recall_score() para verificar se os dados de teste que atestaram verdadeiro foram previstos corretamente em relação a todos os dados que atestaram verdadeiro de forma incorreta:

In [24]:
print(recall_score(y_test, preds))

0.998109640831758


É possível observar que a precisão atestada é de 99,81%, o que sugere que em 99,81% das vezes a árvore de decisão é capaz de prever os dados classificados como verdadeiros corretamente. Por fim, utiliza-se o método hamming_loss() para verificar a fração de previsões que foram realizadas de forma incorreta:

In [25]:
print(hamming_loss(y_test, preds))

1.2123039361927541e-05


Percebe-se que em aproximadamente 0.0012% das vezes a árvore de decisão realiza previsões incorretas.